In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import itertools

In [2]:
baseUrl = 'http://www.10000recipe.com/recipe/'

In [42]:
def PageCrawler(recipeUrl):
    url = baseUrl + recipeUrl

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    recipe_img = [] #레시피 이미지 url
    recipe_title = []  # 레시피 제목
    recipe_source = {}  # 레시피 재료
    recipe_step = [] #레시피 순서

    try:
        res = soup.find("div",class_="centeredcrop")
        imgUrl = res.find("img")["src"]
        res = soup.find('div', 'view2_summary')
        res = res.find('h3')
        recipe_title.append(res.get_text())
        res = soup.find('div', 'view2_summary_info')
        recipe_title.append(res.get_text().replace('\n', ''))
        res = soup.find('div', 'ready_ingre3')
    except(AttributeError):
        return

    # 재료 찾는 for문 가끔 형식에 맞지 않는 레시피들이 있어 try/ except 해준다
    try:
        for n in res.find_all('ul'):
            source = []
            title = n.find('b').get_text()
            recipe_source[title] = ''
            for tmp in n.find_all('li'):
                tempSource = tmp.get_text().replace('\n', '').replace('', '')
                source.append(tempSource.split("    ")[0])

            recipe_source[title] = source
    except (AttributeError):
        return
    
    # 요리순서 찾는 for문
    res = soup.find('div','view_step')
    i = 0
    for n in res.find_all('div','view_step_cont') :
        i = i + 1
        recipe_step.append('#' + str(i) + ' ' + n.get_text().replace('\n','').replace('',''))   
        
    recipe_all = [ imgUrl,recipe_title, recipe_source,recipe_step]  #이미지 url, 제목, 재료, 순서
    return recipe_all

In [43]:
PageCrawler('6900101')

['https://recipe1.ezmember.co.kr/cache/recipe/2018/11/13/39fbf1697aed3e36a2a9b9182e7fbb1b1.jpg',
 ['소고기야채죽 ★ 쫄지마! 다 너의 뜻대로 될꺼니깐!', '2인분 30분 이내 중급 '],
 {'[필수재료]': ['소고기', '양파', '애호박', '당근', '표고버섯', '다시마 육수', '참기름', '소금', '밥'],
  '[밑간재료]': ['소금', '후추', '참기름', '맛술']},
 ['#1 양파, 애호박, 당근, 표고버섯은 다진다.',
  '#2 소고기는 밑간한다.',
  '#3 달군 냄비에 참기름을 두르고 밑간한 소고기를 넣어 볶는다.',
  '#4 밥을 넣어 볶은 후 다시마육수를 붓고 중불로 끓인다.',
  '#5 다진 채소를 넣고 소금 약간을 넣어 간한 후 통깨를 뿌려 완성한다.']]